In [107]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO

In [34]:
q = '''
select 
to_char(dateadd(DAY, -8, getdate()), 'yyyy-mm-dd') as from,
to_char(dateadd(DAY, -1, getdate()), 'yyyy-mm-dd') as to
'''
pd.read_sql(q, con=bun_dw)

,from,to
0,2021-06-08,2021-06-15


In [68]:
# 최근 일주일간 상품등록 갯수 
q = '''
SELECT uid,
       count(DISTINCT id) AS p_cnt
FROM service1_quicket.product_info
WHERE create_date >= to_char(dateadd(DAY, -8, getdate()), 'yyyy-mm-dd') 
  AND create_date <= to_char(dateadd(DAY, -1, getdate()), 'yyyy-mm-dd')
GROUP BY 1'''
p_cnt = pd.read_sql(q, con=bun_dw)

In [36]:
p_cnt['p_cnt'].describe()

count    155380.000000
mean          3.618773
std          11.762255
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         350.000000
Name: p_cnt, dtype: float64

In [37]:
p_cnt.shape

(155380, 2)

In [38]:
p_cnt[p_cnt['p_cnt'] >= 10].shape

(8585, 2)

In [42]:
uid_list = p_cnt['uid'].astype(str).tolist()
uids = ','.join('\'' + str(u) + '\'' for u in uid_list)

In [69]:
q = f'''
SELECT uid,
       sum(up_plus_purchase_cnt) AS up_plus_purchase_cnt,
       sum(up_plus_cnt) AS up_plus_cnt,
       sum(pay_point) AS pay_point,
       sum(total_point) AS total_point
FROM (
        (SELECT UID,
                count(*) AS up_plus_purchase_cnt,
                sum(qty) AS up_plus_cnt,
                sum(pay_point) AS pay_point,
                sum(pay_point) + sum(pay_free) AS total_point
         FROM ad_up_plus
         WHERE created_at >= date_format(DATE_sub(NOW(), INTERVAL 8 DAY), '%Y-%m-%d')
           AND created_at <= date_format(DATE_sub(NOW(), INTERVAL 1 DAY), '%Y-%m-%d')
         GROUP BY 1)
      UNION
        (SELECT UID,
                count(*) AS up_plus_purchase_cnt,
                sum(charge_amount) AS up_plus_cnt,
                sum(CASE
                        WHEN charge_amount = 5 THEN 2500
                        WHEN charge_amount = 10 THEN 4900
                        WHEN charge_amount = 20 THEN 8900
                        WHEN charge_amount = 50 THEN 20000
                        WHEN charge_amount = 100 THEN 40000
                    END) AS pay_point,
                sum(CASE
                        WHEN charge_amount = 5 THEN 2500
                        WHEN charge_amount = 10 THEN 4900
                        WHEN charge_amount = 20 THEN 8900
                        WHEN charge_amount = 50 THEN 20000
                        WHEN charge_amount = 100 THEN 40000
                    END) AS total_point
         FROM store_purchases
         WHERE created_at >= date_format(DATE_sub(NOW(), INTERVAL 8 DAY), '%Y-%m-%d')
           AND created_at <= date_format(DATE_sub(NOW(), INTERVAL 1 DAY), '%Y-%m-%d')
           AND status = 4
         GROUP BY 1)) t
GROUP BY 1'''
up_plus = pd.read_sql(q, con=mysql)

In [54]:
up_plus

,UID,up_plus_purchase_cnt,up_plus_cnt,pay_point,total_point
0,33500,1.0,50.0,20000.0,20000.0
1,40162,1.0,5.0,2500.0,2500.0
2,51415,1.0,20.0,8900.0,8900.0
3,56401,2.0,25.0,11400.0,11400.0
4,59352,1.0,10.0,4900.0,4900.0
...,...,...,...,...,...
723,76771912,1.0,100.0,40000.0,40000.0
724,76773705,1.0,5.0,2500.0,2500.0
725,76774181,1.0,5.0,2500.0,2500.0
726,76783134,1.0,10.0,4900.0,4900.0


In [55]:
up_plus.describe()

,UID,up_plus_purchase_cnt,up_plus_cnt,pay_point,total_point
count,7.280000e+02,728.00000,728.000000,728.000000,728.000000
mean,2.253799e+07,1.48489,42.815934,17297.442308,17834.615385
std,2.959517e+07,1.50553,94.286412,37196.133829,37734.304382
min,3.350000e+04,1.00000,5.000000,0.000000,2500.000000
25%,3.851316e+06,1.00000,5.000000,2500.000000,2500.000000
50%,8.106235e+06,1.00000,10.000000,4900.000000,4900.000000
75%,1.274457e+07,1.00000,50.000000,20000.000000,20000.000000
max,7.678368e+07,22.00000,1600.000000,640000.000000,640000.000000


In [64]:
up_plus[up_plus['pay_point']>=10000]

,UID,up_plus_purchase_cnt,up_plus_cnt,pay_point,total_point
0,33500,1.0,50.0,20000.0,20000.0
3,56401,2.0,25.0,11400.0,11400.0
5,60013,1.0,100.0,40000.0,40000.0
6,66620,2.0,200.0,80000.0,80000.0
7,87724,1.0,50.0,20000.0,20000.0
...,...,...,...,...,...
702,76699259,1.0,50.0,20000.0,20000.0
706,76727468,6.0,300.0,122700.0,122700.0
714,76747461,1.0,50.0,20000.0,20000.0
717,76760075,1.0,50.0,20000.0,20000.0


In [70]:
q = '''
SELECT uid,
       count(DISTINCT id) AS ad_id_cnt,
       sum(total_budget) AS total_budget,
       avg(paid_budget) AS avg_paid_budget,
       sum(paid_budget) AS total_paid_budget,
       sum(free_budget) AS total_free_budget
FROM service1_quicket.ad_set
WHERE created_at >= to_char(dateadd(DAY, -8, getdate()), 'yyyy-mm-dd') 
  AND created_at <= to_char(dateadd(DAY, -1, getdate()), 'yyyy-mm-dd')
GROUP BY 1
'''
sa = pd.read_sql(q, con=bun_dw)

In [60]:
sa.describe()

,uid,ad_id_cnt,total_budget,avg_paid_budget,total_paid_budget,total_free_budget
count,8.310000e+02,831.000000,8.310000e+02,8.310000e+02,8.310000e+02,8.310000e+02
mean,2.318527e+07,2.806258,1.020771e+05,3.581741e+04,9.196356e+04,1.011353e+04
std,3.037968e+07,7.849626,5.016391e+05,9.812574e+04,4.205899e+05,1.045658e+05
min,1.469200e+04,1.000000,2.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.647374e+06,1.000000,9.320000e+03,7.000000e+03,8.960000e+03,0.000000e+00
50%,7.428203e+06,1.000000,2.096500e+04,1.145000e+04,1.996500e+04,0.000000e+00
75%,1.307363e+07,2.000000,5.525000e+04,2.558200e+04,5.222750e+04,1.800000e+03
max,7.678313e+07,172.000000,1.233258e+07,1.400000e+06,9.909091e+06,2.423489e+06


In [74]:
sa[(sa['avg_paid_budget'] >= 20000) & (sa['ad_id_cnt'] >= 2)]

,uid,ad_id_cnt,total_budget,avg_paid_budget,total_paid_budget,total_free_budget
0,3947717,172,3123375,17514,3012510,110865
1,75195397,8,76505,9563,76505,0
2,14692,10,20000,2000,20000,0
3,8700331,3,113190,34730,104190,9000
6,76426479,5,89260,17276,86380,2880
...,...,...,...,...,...,...
777,76541225,2,15750,7875,15750,0
789,11158176,2,75000,37045,74090,910
794,2678704,3,191205,59898,179695,11510
801,60013,3,1065000,338333,1015000,50000


In [142]:
paid_up['uid'] = paid_up['uid'].astype(str)
sa['uid'] = sa['uid'].astype(str)
p_cnt['uid'] = p_cnt['uid'].astype(str)
candidate = pd.merge(p_cnt[p_cnt['p_cnt'] >= 5], pd.merge(paid_up[paid_up['paid_up_cnt'] >= 5], sa[(sa['avg_paid_budget'] >= 1000)], on='uid', how='outer'), on='uid')


In [143]:
candidate

,uid,p_cnt,paid_up_cnt,ad_id_cnt,total_budget,avg_paid_budget,total_paid_budget,total_free_budget
0,9003864,62,28.0,NaN,NaN,NaN,NaN,NaN
1,1109123,25,15.0,NaN,NaN,NaN,NaN,NaN
2,6944993,74,10.0,NaN,NaN,NaN,NaN,NaN
3,2575110,75,NaN,7.0,240000.0,34285.0,240000.0,0.0
4,75942806,232,NaN,1.0,70000.0,65000.0,65000.0,5000.0
...,...,...,...,...,...,...,...,...
441,6222937,8,NaN,2.0,21000.0,10050.0,20100.0,900.0
442,9510107,7,5.0,NaN,NaN,NaN,NaN,NaN
443,8182612,12,19.0,NaN,NaN,NaN,NaN,NaN
444,76706759,6,NaN,1.0,55680.0,52680.0,52680.0,3000.0


## 6월8일 상품 등록 유저 대상 스코어링 후, 파워셀러 분류 작업 결과

In [84]:
import os
os.listdir()

['파워셀러 스코어링.ipynb',
 '.DS_Store',
 'exclude_uid.csv',
 '파워셀러 분석-비교.ipynb',
 'power_seller_score.csv',
 'uids.txt',
 'power_non_power.csv',
 'powerSeller_list copy.csv',
 '파워셀러 분석.ipynb',
 'powerSeller.xlsx',
 'powerSeller copy.xlsx',
 '.ipynb_checkpoints',
 'power_seller_score_3.csv',
 'power_seller_score_2.csv',
 'powerSeller_list.csv']

In [85]:
group = pd.read_csv('power_non_power.csv')


In [89]:
group.columns = ['uid', 'score', 'result', 'etc', 'etc2']

In [91]:
group.result.unique()

array(['파워상점', '일반상점', '보류', nan], dtype=object)

In [92]:
power_seller = group[group['result'] == '파워상점']
normal_seller = group[group['result'] == '일반상점']

In [93]:
print('power seller:', power_seller.shape[0])
print('normal seller:', normal_seller.shape[0])

power seller: 283
normal seller: 258


In [114]:
power_seller.describe()

,uid,score
count,2.830000e+02,283.000000
mean,3.433959e+07,4.163255
std,3.306913e+07,1.094205
min,1.219130e+05,2.727872
25%,7.576932e+06,2.963262
50%,1.219462e+07,5.054110
75%,7.558103e+07,5.130760
max,7.674216e+07,6.465884


In [115]:
normal_seller.describe()

,uid,score
count,2.580000e+02,258.000000
mean,3.834766e+07,3.718402
std,3.414331e+07,1.059697
min,5.371400e+04,2.727616
25%,7.050673e+06,2.865571
50%,1.276824e+07,3.047926
75%,7.581014e+07,5.063031
max,7.674346e+07,5.959382


In [95]:
candidate[candidate['uid'].isin(power_seller['uid'])]

,uid,p_cnt,up_plus_purchase_cnt,up_plus_cnt,pay_point,total_point,ad_id_cnt,total_budget,avg_paid_budget,total_paid_budget,total_free_budget
32,76541225,11,1.0,100.0,40000.0,40000.0,2,15750,7875,15750,0


In [96]:
candidate[candidate['uid'].isin(normal_seller['uid'])]

,uid,p_cnt,up_plus_purchase_cnt,up_plus_cnt,pay_point,total_point,ad_id_cnt,total_budget,avg_paid_budget,total_paid_budget,total_free_budget


## 6월 15일 상품 등록 유저 대상 스코어링

In [110]:
q = f'''
SELECT uid,
       count(CASE
                 WHEN status = 0 THEN 1
             END) AS live_cnt,
       count(DISTINCT id) AS p_cnt
FROM service1_quicket.product_info
WHERE uid IN ({uids})
GROUP BY 1
'''
product = pd.read_sql(q, con=bun_dw)

In [120]:
product['uid'] = product['uid'].astype(str)
p_cnt['uid'] = p_cnt['uid'].astype(str)
p = pd.merge(product, p_cnt, on='uid', how='outer')

In [122]:
print('product shape:', product.shape)
print('p_cnt shape:', p_cnt.shape)

product shape: (155380, 3)
p_cnt shape: (155380, 2)


In [123]:
p.columns = ['uid', 'live_cnt', 'all_p_cnt', 'p_cnt_in_week']

In [124]:
p.head()

,uid,live_cnt,all_p_cnt,p_cnt_in_week
0,5376067,3365,35747,239
1,3336826,57,232,1
2,5434086,72,7182,87
3,12161813,596,1323,46
4,3623936,98,259,4


In [127]:
## 유료 UP 사용
q = f'''
SELECT uid,
       count(*) AS paid_up_cnt
FROM bun_etl_db.service1_quicket_up_count_history
WHERE register_at >= '2021-06-13 00:00:00'
  AND register_at <= '2021-06-15 23:59:59'
  AND up_type = 2
  AND uid IN ({uids})
GROUP BY 1
'''
paid_up = pd.read_sql(q, con=bun_dw)

In [129]:
paid_up.describe()

,uid,paid_up_cnt
count,1.135000e+03,1135.000000
mean,1.591617e+07,9.172687
std,2.508225e+07,23.024873
min,9.274000e+03,1.000000
25%,2.869743e+06,1.000000
50%,5.719516e+06,3.000000
75%,1.144535e+07,7.000000
max,7.678582e+07,359.000000


In [130]:
paid_up[paid_up['paid_up_cnt'] >= 10]

,uid,paid_up_cnt
0,8139962,14
1,2829097,141
2,2487730,67
3,2653973,50
5,76136980,12
...,...,...
1067,76694593,18
1068,75397672,30
1070,1061325,21
1083,76402316,10


In [117]:
q = f'''
SELECT uid,
       min(extract(hour
                   FROM create_date)) AS earliest,
       max(extract(hour
                   FROM create_date)) AS latest,
       latest - earliest AS diff
FROM service1_quicket.product_info
WHERE UID IN ({uids})
  AND create_date BETWEEN '2021-06-13 00:00:00' AND '2021-06-15 23:59:59'
GROUP BY 1
'''
upload_time = pd.read_sql(q, con=bun_dw)

In [111]:
q = f'''
SELECT a.uid,
       a.top_category,
       a.p_cnt_in_top_category,
       b.distinct_category_cnt
FROM
  (SELECT UID,
          category AS top_category,
          p_cnt_in_category AS p_cnt_in_top_category
   FROM
     (SELECT UID,
             left(category_id, 3) AS category,
             count(DISTINCT id) AS p_cnt_in_category,
             row_number() OVER (PARTITION BY UID
                                ORDER BY count(DISTINCT id) DESC) AS ROW
      FROM service1_quicket.product_info
      WHERE UID IN ({uids}) AND status = 0
      GROUP BY 1,
               2)
   WHERE ROW = 1) a
JOIN
  (SELECT UID,
          count(DISTINCT left(category_id, 3)) AS distinct_category_cnt
   FROM service1_quicket.product_info
   WHERE UID IN ({uids})
   GROUP BY 1) b ON a.uid = b.uid
'''
category = pd.read_sql(q, con=bun_dw)

In [118]:
q = f'''
SELECT content_owner AS uid,
       count(*) AS total_click_cnt,
       count(DISTINCT content_id) AS p_cnt,
       cast(total_click_cnt AS float)/p_cnt AS avg_click_cnt
FROM bun_log_db.app_event_type_view
WHERE content_owner IN ({uids})
  AND YEAR||MONTH||DAY = '20210615'
GROUP BY 1
'''
click = pd.read_sql(q, con=bun_dw)

In [119]:
q = f'''
SELECT content_owner AS uid,
       count(*) AS total_imp_cnt,
       count(DISTINCT content_id) AS p_cnt,
       cast(total_imp_cnt AS float)/p_cnt AS avg_imp_cnt
FROM bun_log_db.app_event_type_impression
WHERE content_owner IN ({uids})
  AND YEAR||MONTH||DAY = '20210615'
GROUP BY 1
'''
imp = pd.read_sql(q, con=bun_dw)

In [144]:
q = f'''
SELECT id AS uid,
       favorite_count AS follower_cnt,
       review_count,
       grade
FROM service1_quicket.user_
WHERE id IN ({uids})
'''
user = pd.read_sql(q, con=bun_dw)

In [146]:
tables = [p, upload_time, category, up_plus, paid_up, sa, click, imp, user]
for t in tables:
    t['uid'] = t['uid'].astype(str)


In [147]:
df = pd.merge(p, upload_time, on='uid', how='outer')
df = pd.merge(df, category, on='uid', how='outer')
df = pd.merge(df, up_plus, on='uid', how='outer')
df = pd.merge(df, paid_up, on='uid', how='outer')
df = pd.merge(df, sa, on='uid', how='outer')
df = pd.merge(df, click, on='uid', how='outer')
df = pd.merge(df, imp, on='uid', how='outer')
df = pd.merge(df, user, on='uid', how='outer')

In [148]:
df = df.fillna(0)

In [149]:
df

,uid,live_cnt,all_p_cnt,p_cnt_in_week,earliest,latest,diff,top_category,p_cnt_in_top_category,distinct_category_cnt,...,total_free_budget,total_click_cnt,p_cnt_x,avg_click_cnt,total_imp_cnt,p_cnt_y,avg_imp_cnt,follower_cnt,review_count,grade
0,5376067,3365.0,35747.0,239.0,3.0,23.0,20.0,500,3066.0,12.0,...,0.0,1544.0,396.0,3.898990,33775.0,3290.0,10.265957,1068.0,56.0,525.0
1,3336826,57.0,232.0,1.0,18.0,18.0,0.0,900,57.0,5.0,...,0.0,26.0,22.0,1.181818,573.0,105.0,5.457143,5.0,13.0,130.0
2,5434086,72.0,7182.0,87.0,0.0,23.0,23.0,400,70.0,7.0,...,0.0,281.0,72.0,3.902778,6393.0,194.0,32.953608,410.0,389.0,3804.0
3,12161813,596.0,1323.0,46.0,18.0,22.0,4.0,320,558.0,4.0,...,0.0,202.0,104.0,1.942308,4888.0,637.0,7.673469,106.0,30.0,297.0
4,3623936,98.0,259.0,4.0,19.0,21.0,2.0,320,58.0,10.0,...,0.0,208.0,35.0,5.942857,3883.0,145.0,26.779310,11.0,13.0,119.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155781,3355923,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,900.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
155782,74337286,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,5000.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
155783,74367438,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,1150.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
155784,4030872,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [151]:
import numpy as np
df['if_work_hr'] = np.where(((df['diff']>=8) & (df['diff'] <= 10)) | (df['diff'] == 23), 0.5, 0)
df['if_category'] = np.where((df['p_cnt_in_top_category'] >= 10) & (df['distinct_category_cnt'] <= 3), 0.5, 0)

In [152]:
df.columns

Index(['uid', 'live_cnt', 'all_p_cnt', 'p_cnt_in_week', 'earliest', 'latest',
       'diff', 'top_category', 'p_cnt_in_top_category',
       'distinct_category_cnt', 'up_plus_purchase_cnt', 'up_plus_cnt',
       'pay_point', 'total_point', 'paid_up_cnt', 'ad_id_cnt', 'total_budget',
       'avg_paid_budget', 'total_paid_budget', 'total_free_budget',
       'total_click_cnt', 'p_cnt_x', 'avg_click_cnt', 'total_imp_cnt',
       'p_cnt_y', 'avg_imp_cnt', 'follower_cnt', 'review_count', 'grade',
       'if_work_hr', 'if_category'],
      dtype='object')

In [153]:
from sklearn.preprocessing import MinMaxScaler
cols = ['live_cnt', 'p_cnt_in_week',
        'up_plus_purchase_cnt', 'up_plus_cnt', 'pay_point', 'total_point', 'paid_up_cnt', 
        'ad_id_cnt', 'total_paid_budget', 'total_budget', 
        'follower_cnt', 'review_count', 'grade', 
        'avg_click_cnt', 'avg_imp_cnt']
s = df[cols + ['uid', 'if_work_hr', 'if_category']]
s[cols] = MinMaxScaler().fit_transform(s[cols])

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [155]:
def score(live_cnt, 
          p_cnt_in_week, 
          up_plus_purchase_cnt, 
          up_plus_cnt,
          paid_up_cnt,
          total_point, 
          ad_id_cnt, 
          total_budget, 
          follower_cnt, 
          review_count, 
          grade, 
          avg_click_cnt, 
          avg_imp_cnt, 
          if_work_hr, 
          if_category):
    sc = s['live_cnt'] * live_cnt \
    + s['p_cnt_in_week'] * p_cnt_in_week \
    + s['up_plus_purchase_cnt'] * up_plus_purchase_cnt \
    + s['up_plus_cnt'] * up_plus_cnt \
    + s['paid_up_cnt'] * paid_up_cnt \
    + s['total_point'] * total_point \
    + s['ad_id_cnt'] * ad_id_cnt \
    + s['total_budget'] * total_budget \
    + s['follower_cnt'] * follower_cnt \
    + s['review_count'] * review_count \
    + s['grade'] * grade \
    + s['avg_click_cnt'] * avg_click_cnt \
    + s['avg_imp_cnt'] * avg_imp_cnt \
    + s['if_work_hr'] * if_work_hr \
    + s['if_category'] * if_category
    return sc

In [ ]:
score(live_cnt = 5, 
      p_cnt = 1, 
      up_plus_purchase_cnt = 2,
      up_plus_cnt = 3,
      total_point = 3,
      ad_id_cnt = 6,
      total_budget = 3,
      follower_cnt = 4, 
      review_count = 4,
      grade = 3, 
      avg_click_cnt = 5,
      avg_imp_cnt = 2,
      if_work_hr = 5,
      if_category = 5)

In [156]:
s['score'] = score(live_cnt = 5, 
                   p_cnt_in_week = 3, 
                   up_plus_purchase_cnt = 2, 
                   up_plus_cnt = 2,
                   paid_up_cnt = 4,
                   total_point = 2, 
                   ad_id_cnt = 6, 
                   total_budget = 2, 
                   follower_cnt = 4, 
                   review_count = 4, 
                   grade = 3, 
                   avg_click_cnt = 5, 
                   avg_imp_cnt = 2, 
                   if_work_hr = 4, 
                   if_category = 4)

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [159]:
candidate_score = s[s['uid'].isin(candidate['uid'].astype(str))]
non_candidate_score = s[~s['uid'].isin(candidate['uid'].astype(str))]


In [160]:
print('shape of candidate score:', candidate_score.shape)
print('shape of non candidate score:', non_candidate_score.shape)

shape of candidate score: (446, 20)
shape of non candidate score: (155340, 20)


In [161]:
candidate_score['score'].describe()

count    446.000000
mean       1.739686
std        1.615301
min        0.079179
25%        0.441126
50%        1.178673
75%        2.607184
max       13.738523
Name: score, dtype: float64

In [162]:
non_candidate_score['score'].describe()

count    155340.000000
mean          0.227909
std           0.544283
min           0.008784
25%           0.033205
50%           0.066759
75%           0.136646
max          11.426128
Name: score, dtype: float64

In [169]:
candidate_score.sort_values('score', ascending=False)[['uid', 'score']].to_csv('candidate_score.csv', index=False)

In [166]:
candidate_score.sort_values('score', ascending=False)[['uid', 'score']][:30]

,uid,score
2234,3947717,13.738523
12989,592959,10.017732
2702,6972746,7.734288
11726,4673979,7.620157
14198,216373,7.155592
4153,9503002,7.022237
91136,12092136,6.645026
21128,4779682,6.522073
19520,76736772,6.337716
6198,6836663,5.424041


In [ ]:
q = '''
select 
to_char(dateadd(DAY, -2, getdate()), 'yyyy-mm-dd') as from,
to_char(dateadd(DAY, -1, getdate()), 'yyyy-mm-dd') as to
'''
pd.read_sql(q, con=bun_dw)

In [32]:
q = '''
SELECT flag_used
FROM service1_quicket.product_info
WHERE create_date >= to_char(dateadd(DAY, -2, getdate()), 'yyyy-mm-dd')
  AND create_date <= to_char(dateadd(DAY, -1, getdate()), 'yyyy-mm-dd')
GROUP BY 1
'''
flag_used = pd.read_sql(q, con=bun_dw)

In [33]:
flag_used

,flag_used
0,2
1,1


In [29]:
# 업플러스 인앱결제
q = '''
SELECT uid,
       sum(charge_amount) AS charge_cnt
FROM store_purchases
WHERE status = 4
  AND created_at >= date_format(DATE_sub(NOW(), INTERVAL 8 DAY), '%Y-%m-%d')
  AND created_at <= date_format(DATE_sub(NOW(), INTERVAL 1 DAY), '%Y-%m-%d')
GROUP BY 1
'''
inapp = pd.read_sql(q, con=mysql)

In [30]:
inapp

,uid,charge_cnt
0,33500,50.0
1,40162,5.0
2,51415,20.0
3,56401,25.0
4,59352,10.0
...,...,...
547,76768145,5.0
548,76769326,10.0
549,76769474,5.0
550,76771912,100.0
